<img
    src="https://upload.wikimedia.org/wikipedia/commons/4/42/CNAM_Logo.svg" 
    alt=""
    height="200px" 
    width="200px"
    align=left
/> 

<center> <br>
  <h1 style="color:#7c7979";></h1>
</center>  

<center>
  <h1 style="color:#000000";>Nettoyer un jeu de données avec Pandas en Python</h1>
</center> 

Notebook inspiré de [Jean-Nicholas Hould](http://www.jeannicholashould.com/) et de [Hadley Wickham](http://vita.had.co.nz/papers/tidy-data.pdf)

Lorsque l'on s'attaque à l'analyse d'un jeu de données, celui-ci est rarement "prêt à l'emploi", et bien souvent il faut procéder à un nettoyage plus ou moins important des données, aussi bien sur la forme (apparence) que sur le fond (structure).

In [ ]:
import pandas as pd
import datetime
import glob
import re

In [ ]:
pd.options.display.max_rows = 10

## I - Premiers nettoyages concernant la mise en forme du jeu de données 

Dans la première partie de ce cours nous allons nous intéresser au nettoyage "de forme" du jeu de données contenu dans le fichier excel `chomage.xls`. Ce jeu de données correspond au taux de chômage trimestriel par sexe et âge (en %).

In [ ]:
chom = pd.read_excel('chomage.xls')
chom

In [ ]:
chom.shape

$\rightarrow$ Comme on peut le remarquer, ce jeu de données n'est pas du tout prêt à être manipulé et analysé. Une importante phase de nettoyage s'impose.

### 1 - Retrait de lignes/colonnes superflues

La fonction `read_excel` de pandas possède de nombreux arguments pour formater le chargement d'un dataframe à partir d'un fichier .xls

#### a - Retrait des lignes superflues

* Le retrait des **lignes d'en-tête** se fait avec l'argument `skiprows`. La valeur attendue est le nombre de lignes d'en-tête du fichier à retirer.

* Le retrait des **lignes de bas de page** se fait avec l'argument `skip_footer`. La valeur attendue est le nombre de lignes de bas de page du fichier à retirer.

In [ ]:
chom = pd.read_excel('chomage.xls', skiprows=7, skip_footer=1)
chom

#### b - Retrait de colonnes, sélection des colonnes à garder

* L'argument `usecols` permet de spécifier l'ensemble des colonnes à conserver. La valeur attendue peut-être :
    * une liste d'entiers : dans ce cas cette liste indique la liste des colonnes à prendre en compte
    * une chaîne de caractères comportant les lettres des colonnes Excel à prendre en compte, séparées pas des virgules ou par ":" pour les ranges (ex: “A:E” ou “A,C,E:F”)

In [ ]:
cols = [x for x in range (chom.shape[1]-1)] # retrait de la dernière colonne, inutile ici
chom = pd.read_excel('chomage.xls', skiprows=7, skip_footer=1, usecols=cols)
chom

### 2 - Renommer les colonnes

In [ ]:
age_labels = ["15-24","25-49",">49","Total"]
sexe_labels = ["Hommes", "Femmes", "TOTAL"]

new_col_names = ["Trimestres"]
for i in range (len(sexe_labels)):
    for j in range (len(age_labels)):
        new_col_names.append(sexe_labels[i] + " " + age_labels[j])
print(new_col_names)

In [ ]:
chom.columns = new_col_names
chom

### 3 - S'assurer que les valeurs ont le bon format/type

Ici on remarque que dans la première colonne "Hommes 15-24", le symbole % a été inséré dans certaines cellules. Or ces cellules ne sont censées contenir que des données numériques, aucun caractère non-numérique ne doit y figurer.

Pour retirer le symbôle `%` on utilise a fonction `str.replace()`. 

**Attention** : Dans la colonne "Hommes 15-24" il y a à la fois des valeurs de type `str` (celles avec le %) et des valeurs de type `float` (celles qui n'ont pas le %). Il faut donc d'abord tout convertir en `str` puis appliquer la fonction `str.replace()` et enfin tout reconvertir en `float`.

In [ ]:
chom["Hommes 15-24"] = chom["Hommes 15-24"].astype(str)
chom["Hommes 15-24"] = chom["Hommes 15-24"].str.replace("%", "")
chom["Hommes 15-24"] = chom["Hommes 15-24"].str.replace(",", ".")
chom["Hommes 15-24"] = chom["Hommes 15-24"].astype(float)
chom

### 4 - Gérer les valeurs manquantes

Pandas utilise la valeur `NaN` (Not a Number) pour réprésenter les valeurs manquantes dans un jeu de données.

Une fonction Pandas que l'on peut utiliser pour savoir si certaines données sont manquantes dans le Dataframe est la fonction `isnull()` : cette fonction retourne `True` lorsque la valeur testée est manquante (`NaN`) et `False` sinon. 

In [ ]:
chom.isnull()

#### a - Filtrer les valeurs manquantes

Dans un premier temps, il peut être souhaitable de filtrer/retirer les valeurs manquantes dans le dataframe.

Pour cela on utilise la fonction `dropna()`. Cette fonction possède plusieurs arguments, permettant de moduler le niveau de filtrage des valeurs manquantes :
* `axis=` (0 ou 1) : filtrage des valeurs manquantes selon les lignes (axis = 0) ou les colonnes (axis=1). Valeur par défaut : axis=0.
* `how=all` : argument permettant de ne retirer que les lignes (pour axis=0) ou toutes les colonnes (pour axis=1) du dataframe pour lequelles **toutes** les valeurs sont manquantes. 

$\rightarrow$ **Attention** : si cet argument n'est pas spécifié, chaque ligne (resp. chaque colonne) contenant **au moins une** donnée manquante sera supprimée !
* `inplace=True` : applique directement la modification dans le dataframe (sans créer un autre dataframe).
* `thresh=` (nombre entier *n*) : permet de ne conserver que les lignes (resp. les colonnes) contenant *au moins n* valeurs non manquantes.

In [ ]:
chom.dropna(how='all', inplace=True)
chom

$\rightarrow$ Auncun ligne n'a été supprimée (même celle correspondant à l'observation "T4.1996"), car la colonne "Trimestres" possède toujours une valeur.  Pour pouvoir supprimer les lignes sans intérêt comme par exemple "T4.1996" on pourra alors utiliser l'argument `thresh` :

In [ ]:
chom.dropna(thresh=12, inplace=True)
chom

$\rightarrow$ La ligne correspondant à l'observation "T4.1996" a bien été supprimée, ainsi que "T2.1996" (elle ne contenait que 11 valeurs non manquantes et non 12).

#### b - Remplacer les valeurs manquantes

Plutôt que de retirer les valeurs manquantes (et potentiellement supprimer aussi des valeurs existantes), on peut aussi vouloir remplacer les valeurs manquantes par n'importe quel nombre. 

Pour cela il existe en Pandas la fonction `fillna()`. Cette fonction peut être appelée de différentes manières :
* en lui donnant une valeur chiffrée : dans ce cas, chaque valeur manquante du dataframe sera remplacée par cette valeur chiffrée
* en lui donnant un dictionnaire où les clés seront les noms des colonnes et les valeurs du dictionnaire seront les valeurs chiffrées qui remplaceront les `NaN` : cela permet de spécifier une valeur de remplacement distincte pour chaque colonne

$\underline{Remarque}$ : il existe de nombreuses fonctionnalités avec la fonction `fillna()`, comme par exemple celle de remplacer des valeurs manquantes par la moyenne de la colonne, de la ligne ou toute autre sous-partie du dataframe.

In [ ]:
chom.fillna({'Hommes 25-49': 0, 'Femmes 25-49': 0.5}, inplace=True)
chom

$\rightarrow$ Les valeurs manquantes de la colonne 'Hommes 25-49' sont remplacées par 0 et celles de la colonne 'Femmes 25-49' sont remplacées par 0.5.

In [ ]:
chom.fillna(0, inplace=True)
chom

$\rightarrow$ Toutes les valeurs manquantes restantes dans le dataframe sont remplacées par 0.

### 5 - Retirer les données dupliquées

Des lignes dupliquées peuvent être présentes dans un jeu de données. 

La fonction `duplicated()` de Pandas retourne pour chaque ligne un booléen indiquant si la ligne en question est un duplicata d'une ligne précédente :

In [ ]:
chom.duplicated()

$\rightarrow$ La dernière ligne est un duplicata (c'est un duplicata de l'avant dernière).

Pour supprimer les lignes dupliquées on utilise la fonction `drop_duplicates()` :

In [ ]:
chom.drop_duplicates()

$\rightarrow$ La dernière ligne qui était un duplicata de l'avant dernière a bien été supprimée.

## II - Transformations du jeu de données

Une fois le jeu de données nettoyé du point de vue de sa forme, il reste à le nettoyer du point de vue de sa structure. Mieux le jeu de données sera structuré, plus aisées seront les analyses que l'on effectuera ensuite.

Pour cela il est important de rappeler les 3 **règles qui définissent un jeu de données "propre et ordonné"** :
* Chaque variable représente une *unique* colonne et contient des valeurs
* Chaque observation forme une ligne
* Chaque type d'unité d'observation représente un dataframe

Rappels de quelques **définitions** :

* *variable* : intitulé d'une mesure ou d'un attribut. *ex : taille, poids, sexe, etc*.
* *valeur* : la mesure ou l'attribut en question. *ex : 152 cm, 80 kg, femme, etc*.
* *observation* : toutes les entitées mesurées sur la même unité. *ex : chaque personne*.


Voici 4 types de **dataframes désordonnés** que nous allons étudier et nettoyer :
* 1) Les intitulés des colonnes sont des valeurs et pas des noms
* 2) Plusieurs variables sont stockées dans une colonne
* 3) Certaines variables sont stockées à la fois dans des lignes et des colonnes
* 4) Un même type d'unité d'observation est stocké dans différents dataframes


### 1 - Les intitulés des colonnes sont des valeurs et pas des noms

Prenons ce jeu de données issu du Pew Research Center, donnant la répartition des salaires selon les religions.

In [ ]:
df = pd.read_csv("religion.csv")
df

Le problème de ce dataframe est que les intitulés des colonnes sont précisément des valeurs, en l'occurence ce sont des valeurs possibles de salaire. Cet agencement des données n'est pas du tout favorable aux analyses, notamment aux études de corrélation etc ...

Pour ordonner ce dataframe il faut en quelques sortes faire pivoter les données de telle sorte à ce que l'on passe un format "large" (en colonnes) à un format "long" (en lignes). On utilise pour cela la fonction `melt()` de Pandas.

Les arguments à spécifier sont les suivants :
* `id_vars` : colonne(s) à utiliser comme "référant". Ici c'est la colonne "religion".
* `value_vars` : colonne(s) à faire pivoter. Par défaut toutes les colonnes qui ne sont pas désignées comme `id_vars` sont pivotées. Ici c'est le cas, donc on ne spécifiera pas cet argument.
* `var_name` : nom à donner à la colonne obtenue après pivot. Ici on pourra l'appeler "salaire".
* `value_name` : nom à donner à la colonne contenant les valeurs. Ici les valeurs sont les fréquences pour chaque tranche de salaire, donc on pourra appeler cette colonne de valeurs "freq".

In [ ]:
formatted_df = pd.melt(df, id_vars=["religion"], var_name="salaire", value_name="freq")
formatted_df

$\rightarrow$ Le dataframe est classé en fonction des différentes valeurs de la colonne "salaire".

In [ ]:
formatted_df = formatted_df.sort_values(by=["religion"])
formatted_df

### 2 - Plusieurs variables sont stockées dans une colonne

Prenons un jeu de données de l'organisation mondiale de la santé concernant des cas de tuberculose.

Voici les éléments qui font de ce dataframe un jeu non ordonné :
- Les colonnes commençant par "m" ou "f" contiennent plusieurs variables :
    - Sexe ("m" ou "f")
    - Groupe d'âge ("0-14","15-24", "25-34", "45-54", "55-64", "65", "unknown")
- Mélange de 0 et de valeurs manquantes ("NaN")

In [ ]:
tub = pd.read_csv("tuberculose.csv")
tub

* Plusieurs étapes sont nécessaires pour aboutir à un jeu de données propre. On commence par appliquer un `melt()` sur les colonnes qui combinent sexe et âge : 

In [ ]:
tub = pd.melt(tub, id_vars=["country","year"], var_name="sex_and_age", value_name="cases", )
tub

* A partir de la colonne "sex_and_age" on extrait dans un dataframe 3 sous-colonnes qui contiendront respectivement le sexe, la borne d'âge inférieure et la borne d'âge supérieure.

Pour l'extraction de caractères on peut utiliser la fonction `str.extract()` de Pandas avec l'argument `expand=False` pour signifier que l'extraction ne se fait pas dans le dataframe d'origine.

La syntaxe `"(\D)(\d+)(\d{2})"` donnée en argument de la fonction `str.extract()`, provient de la *Regular Expression Syntax* de python (https://docs.python.org/2/library/re.html):
* \D : signifie que l'on extrait le caractère rencontré s'il s'agit d'un caractère non-numérique : ici on extrait la lettre qui correspond au sexe ("m" ou "f")
* \d : signifie que l'on extrait le caractère rencontré s'il s'agit d'un caractère numérique (digit).
* \d{2} : signifie que l'on extrait les 2 prochains caractères numériques : ici on extrait automatiquement ensemble les deux derniers digits.
* \d+ : signifie qu'à partir d'un caractère numérique rencontré on extrait aussi tous ceux qui suivent si ce sont aussi des digits : ici on extrait le(s) premier(s) digit(s) après la lettre et avant les deux derniers digits.

In [ ]:
tmp_tub = tub["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=False)  
tmp_tub

* On renomme ces 3 sous-colonnes :

In [ ]:
tmp_tub.columns = ["sex", "age_lower", "age_upper"]
tmp_tub

* On crée une 4ème colonne qui réunit la borne inférieure et supérieur d'âge, séparées par un tiret :

In [ ]:
tmp_tub["age"] = tmp_tub["age_lower"] + "-" + tmp_tub["age_upper"]
tmp_tub

* On concatène le dataframe d'origine avec cette 4ème colonne que l'on vient de créer :

In [ ]:
tub = pd.concat([tub, tmp_tub], axis=1)
tub

* On retire les colonnes inutiles avec la fonction `drop()` de Pandas, à savoir les colonnes "sex_and_age", "age_lower" et "age_upper" :

In [ ]:
tub = tub.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
tub

* On supprime les lignes pour lesquelles il y a des données manquantes :

In [ ]:
tub = tub.dropna()
tub

* On peut classer le dataframe par ordre croissant en fonction du pays, puis de l'année, puis du sexe et enfin de l'âge : 

In [ ]:
tub = tub.sort_values(ascending=True,by=["country", "year", "sex", "age"])
tub

* Enfin, on peut re-définir l'ordre des colonnes avec la fonction `reindex()` :

In [ ]:
ordre_colonnes = ["country", "year", "sex", "age", "cases"]
tub = tub.reindex(columns=ordre_colonnes)
tub

### 3 - Certaines variables sont stockées à la fois dans des lignes et des colonnes

Prenons un jeu de données du Global Historical Climatology Network qui représente les mesures météo quotidiennes prises au niveau de la station MX17004 au Mexique pendant 5 mois en 2010.

Les problèmes de ce jeu de données sont les suivants : 
* les variables sont à la fois stockées dans des lignes ("tmin", "tmax") et des colonnes ("days").
* il y a autant de colonnes que de jours

In [ ]:
wth = pd.read_csv("weather.csv")
wth

* On commence là encore par appliquer un `melt()` sur les colonnes qui représentent les jours (= on fait pivoter les jours) : 

In [ ]:
wth = pd.melt(wth, id_vars=["id", "year","month","element"], 
              var_name="day")
wth

* Au lieu d'avoir d1, d2, etc ... on extrait seulement le numéro du jour avec la fonction `str.extract` vue précédemment.

*Remarque* : la colonne "id" ayant toujours la même valeur, on force son contenu à être toujours le même, c'est à dire à être la chaîne de caractères "MX17004".

In [ ]:
wth["day"] = wth["day"].str.extract("(\d+)", expand=False) 
wth["id"] = "MX17004"
wth

* On convertit les éléments des colonnes "year", "month" et "day" en nombres entiers :

In [ ]:
wth["year"] = wth["year"].astype(int)
wth["month"] = wth["month"].astype(int)
wth["day"] = wth["day"].astype(int)
wth

* On écrit une fonction qui crée une date à partir d'une année, d'un mois et d'un jour. Puis on crée une nouvelle colonne dans le dataframe en appelant cette fonction :

In [ ]:
def create_date(row):
    return datetime.datetime(year=row["year"], month=row["month"], day=row["day"])

wth["date"] = wth.apply(lambda row: create_date(row), axis=1)
wth

* On supprime du dataframe les colonnes désormais inutiles, à savoir "year", "month" et "day", ainsi que les lignes ne possédant que des valeurs manquantes :

In [ ]:
wth = wth.drop(['year',"month","day"], axis=1)
wth = wth.dropna()
wth

* Enfin, on peut faire pivoter les températures max et min dans des colonnes avec la fonction `pivot_table()` :

In [ ]:
wth = wth.pivot_table(index=["id","date"], columns="element", values="value")
wth.reset_index(drop=False, inplace=True)
wth

### 4 - Un même type d'unité d'observation est stocké dans différents dataframes

Prenons un jeu de données correspondant aux prénoms des bébés garçons donnés dans l'état de l'Illinois entre 2014 et 2015.

Les problèmes de ce jeu de données sont les suivants : 
* Les données sont réparties dans différents fichers/tableaux.
* La variable "année" est présente dans le nom des fichiers (et non dans les tableaux eux-mêmes).

In [ ]:
baby14 = pd.read_csv("2014_baby_names.csv")
baby14

In [ ]:
baby15 = pd.read_csv("2015_baby_names.csv")
baby15

* On commence par stocker dans une variable les noms des différents fichiers qui nous intéressent (ici il y en a 2 en tout). Comme ces deux fichiers sont nommés de façon identique on pourra utiliser le symbôle `*` pour faire référence aux parties qui sont différentes dans leur nomination :

In [ ]:
allFiles = glob.glob("201*_baby_names.csv")
print(allFiles)

* On crée (ou on trouve déjà toute faite) une fonction qui permet d'extraire un groupe de caractères dans le nom d'un fichier : ici on souhaite extraire l'année, c'est à dire les 4 digits :

In [ ]:
def extract_year(string):
    match = re.match("(\d{4})", string) 
    if match != None: 
        return match.group(1)

* On parcourt la liste des noms de fichiers et pour chaque fichier :
    * on crée un dataframe en faisant un `read_csv()`
    * on extrait la date contenue dans le nom du fichier en appelant la petite fonction `extract_year` écrite au-dessus. Puis on crée dans le dataframe une nouvelle colonne appelée "year" qui contient comme valeur l'année extraite (à chaque ligne).
    * on rajoute ce dataframe dans une liste que l'on appelle "df_list"

In [ ]:
df_list= []
for file in allFiles:
    babies = pd.read_csv(file)
    babies["year"] = extract_year(file)
    df_list.append(babies)

* On concatène les dataframes contenus dans la liste "df_list"

In [ ]:
babies = pd.concat(df_list)
babies